In [70]:
# import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

events_df = pd.read_csv('events_data.csv')
user_df = pd.read_csv('user_events.csv')

In [71]:
events_df.head()

,event_id,category,event_name,event_description,event_date,event_time,venue,city,state,country
0,1,Sports,Football Match,Watch two of the best football teams in the wo...,2023-06-10,19:00:00,Wankhede Stadium,Mumbai,Maharashtra,India
1,2,Music,EDM Music Festival,Dance to the beats of the biggest EDM artists ...,2023-06-15,15:00:00,Bangalore Palace Grounds,Bengaluru,Karnataka,India
2,3,Food and Drink,Wine and Cheese Tasting,Sample a variety of fine wines and cheeses,2023-06-25,18:00:00,Taj Mahal Palace Hotel,Mumbai,Maharashtra,India
3,4,Technology,Artificial Intelligence Summit,Explore the latest developments in AI and mach...,2023-07-05,10:00:00,IIT Delhi,New Delhi,Delhi,India
4,5,Arts and Culture,Photography Exhibition,Discover the works of the best photographers i...,2023-07-15,11:00:00,Jehangir Art Gallery,Mumbai,Maharashtra,India


In [72]:
# preprocess the data
events_df['event_description'].fillna('', inplace=True)

In [73]:
# create a user-event matrix for the original dataset
user_event_matrix = user_df.pivot(index='user_id', columns='event_id', values='event_rating').fillna(0)
user_event_matrix = sp.csr_matrix(user_event_matrix.values)

In [74]:
user_event_matrix = user_event_matrix.astype(float)

In [75]:
u, s, vt = svds(user_event_matrix, k=20)

s_diag_matrix = np.diag(s)
predicted_ratings = np.dot(np.dot(u, s_diag_matrix), vt)

In [76]:

# create a document-term matrix for event descriptions
vectorizer = CountVectorizer(stop_words='english')
event_descriptions = vectorizer.fit_transform(events_df['event_description'])
cosine_similarities = cosine_similarity(event_descriptions)

In [77]:

# define a function to get top n similar events based on content
def get_similar_events_content(event_id, n):
    event_index = events_df[events_df['event_id'] == event_id].index[0]
    similar_indices = cosine_similarities[event_index].argsort()[:-n-1:-1]
    similar_events = events_df.iloc[similar_indices]['event_id'].tolist()
    return similar_events

# define a function to get top n recommended events for a user
def get_recommended_events(user_id, n):
    user_ratings = predicted_ratings[user_id - 1]
    event_indices = np.argsort(user_ratings)[::-1]
    recommended_events = []
    for event_index in event_indices:
        if len(recommended_events) == n:
            break
        event_id = events_df.iloc[event_index]['event_id']
        similar_events = get_similar_events_content(event_id, 5)
        for similar_event in similar_events:
            if similar_event not in recommended_events:
                recommended_events.append(similar_event)
            if len(recommended_events) == n:
                break
    return recommended_events

In [81]:

# test the original recommendation system
user_id = 3
n_recommendations = 5
recommended_events = get_recommended_events(user_id, n_recommendations)
print('Recommended events for user', user_id, 'using original model:', recommended_events)

Recommended events for user 3 using original model: [15, 47, 16, 30, 60]
